# In this notebook, we look at a few methods for the advection equation. 

## $u_t+cu_x=0$ is a linear wave equation. 

The solution is given by $u(x,t)=u_0(x-ct)$ where $u_0(x,0)$ is the initial condition. The wave propagates with constant speed $c$ to the left.

In [ ]:
## import packages
import platform
import sys
import time
import numpy as np
import scipy as sp
#import scipy.sparse as spsp # sparse matrix 
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt 
import matplotlib.animation as animation

print("Python 3 version is", sys.version)

print("Matplotlib version is", matplotlib.__version__)

print("Numpy version is", np.__version__)

print("Scipy version is", sp.__version__)

print("System information: ", platform.uname())


In [ ]:
# define space 
L = 6.0
N = 201
x = np.linspace(0,L,N)
dx = L/(N-1)

# define time and CFL safety condition
gamma = 0.8
dt = dx * gamma# try unstable scheme as well 1.05
T = 6.0
ndt = int(T/dt)

# Define time stamp
timestr = time.strftime("%Y%m%d-%H%M%S")

In [ ]:
# define initial condition

u = np.cos(x-3)#3./np.cosh(x-3)**2#
u0 = u

plt.plot(x,u0, 'k')
plt.ylabel('Initial condition')
plt.savefig('initial_condition' + timestr + '.png')

## We look at two FD methods, one with Upwind method, and one with centered difference in space; both forward in time.

Upwind in space: $U_j^{+1}=U_j-c\frac{dt}{dx}(U_j-U_{j-1})$

Centered difference in space: $U_j^{+1}=U_j-c\frac{dt}{2dx}(U_{j+1}-U_{j-1})$. This method is unconditionally unstable (via the Neumann analysis).

In [ ]:
# define matrix forward u_j = u_j - c dt / dx (u_j - u_j-1)

Df = np.eye(N) # spsp.eye(N)

for j in range(N-1):
    Df[j+1,j] = -1
    
# periodic condition
Df[0,-1] = -1

# define matrix centered difference u_j = u_j - .5c dt/dx (u_j+1 - u_j-1)
Dc = np.zeros((N,N))
    
for j in range(N-1):
    Dc[j,j+1] = 1
    Dc[j+1,j] = -1

# periodic condition
Dc[-1,0] = 1
Dc[0,-1] = -1

In [ ]:
# visualization 

# Set up formatting for the movie files
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

fig = plt.figure()

img = []

### In the following, we take the wave speed $c$ to be 1. The upwind method is stable if the CFL condition is satisfied, but the FTCS is unconditionally unstable. 

In the following, run one cell only

In [ ]:
# # # Main time stepping

# for t in range(ndt):
    
# # advection upwind
#     u = u - dt/dx*np.dot(Df,u)
#     img.append(plt.plot(x,u,color="blue"))
# ani = animation.ArtistAnimation(fig,img, interval=50, blit=True, repeat_delay=0)

# ani.save('advection_upwind' + timestr + '.mp4', writer=writer)

    

In [ ]:
# Main time stepping-- FTCS

# only print 35 steps and print max value 
# at each step to see unstable solution

for t in range(35):
    
# advection centered <----- unconditional unstable
    u = u - 0.5*dt/dx*np.dot(Dc,u)
    print(u.max())
    img.append(plt.plot(x,u,color="red"))
ani = animation.ArtistAnimation(fig,img, interval=50, blit=True, repeat_delay=0)

ani.save('advection_centered' + timestr + '.mp4', writer=writer)
